In [8]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hhs1516","key":"df77d1870d62b8065b1ea188acc8273c"}'}

In [9]:
ls -1ha kaggle.json

kaggle.json


In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         99           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        205           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        382           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       2604            True  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      20326            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       5349            True  
connectx

In [11]:
!kaggle competitions download -c lish-moa

 98% 9.00M/9.16M [00:00<00:00, 91.8MB/s]
100% 9.16M/9.16M [00:00<00:00, 84.7MB/s]
  0% 0.00/38.9k [00:00<?, ?B/s]
100% 38.9k/38.9k [00:00<00:00, 34.5MB/s]
  0% 0.00/257k [00:00<?, ?B/s]
100% 257k/257k [00:00<00:00, 83.5MB/s]
  0% 0.00/233k [00:00<?, ?B/s]
100% 233k/233k [00:00<00:00, 71.6MB/s]
  0% 0.00/535k [00:00<?, ?B/s]
100% 535k/535k [00:00<00:00, 176MB/s]
 89% 49.0M/54.8M [00:00<00:00, 77.0MB/s]
100% 54.8M/54.8M [00:00<00:00, 112MB/s] 


In [12]:
!unzip -n /content/test_features.csv.zip
!unzip -n /content/train_features.csv.zip
!unzip -n /content/train_targets_nonscored.csv.zip
!unzip -n /content/train_targets_scored.csv.zip
!unzip -n /content/sample_submission.csv.zip

Archive:  /content/test_features.csv.zip
  inflating: test_features.csv       
Archive:  /content/train_features.csv.zip
  inflating: train_features.csv      
Archive:  /content/train_targets_nonscored.csv.zip
  inflating: train_targets_nonscored.csv  
Archive:  /content/train_targets_scored.csv.zip
  inflating: train_targets_scored.csv  
Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [27]:
!pip install iterstrat

ERROR: Could not find a version that satisfies the requirement iterstrat (from versions: none)
ERROR: No matching distribution found for iterstrat


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm

%matplotlib inline
warnings.filterwarnings("ignore")
plt.style.use("seaborn-whitegrid")

ModuleNotFoundError: ignored

In [14]:
train_features = pd.read_csv("/content/train_features.csv")
train_targets = pd.read_csv("/content/train_targets_scored.csv")
test_features = pd.read_csv("/content/test_features.csv")
submission = pd.read_csv("/content/sample_submission.csv")
#data = pd.concat([df_train, df_test])

In [15]:
def preprocess(df): # one-hot encoding
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

#drop ctl_vehicle
train_targets = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [16]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
                  loss='binary_crossentropy', 
                  )
    return model

In [21]:
from typing import Tuple, List, Callable, Any

from sklearn.utils import check_random_state  # type: ignore
seed = 42
### from eli5
def iter_shuffled(X, columns_to_shuffle=None, pre_shuffle=False,
                  random_state=None):
    rng = check_random_state(random_state)

    if columns_to_shuffle is None:
        columns_to_shuffle = range(X.shape[1])

    if pre_shuffle:
        X_shuffled = X.copy()
        rng.shuffle(X_shuffled)

    X_res = X.copy()
    for columns in tqdm(columns_to_shuffle):
        if pre_shuffle:
            X_res[:, columns] = X_shuffled[:, columns]
        else:
            rng.shuffle(X_res[:, columns])
        yield X_res
        X_res[:, columns] = X[:, columns]



def get_score_importances(
        score_func,  # type: Callable[[Any, Any], float]
        X,
        y,
        n_iter=5,  # type: int
        columns_to_shuffle=None,
        random_state=None
    ):
    rng = check_random_state(random_state)
    base_score = score_func(X, y)
    scores_decreases = []
    for i in range(n_iter):
        scores_shuffled = _get_scores_shufled(
            score_func, X, y, columns_to_shuffle=columns_to_shuffle,
            random_state=rng, base_score=base_score
        )
        scores_decreases.append(scores_shuffled)

    return base_score, scores_decreases



def _get_scores_shufled(score_func, X, y, base_score, columns_to_shuffle=None,
                        random_state=None):
    Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
    res = []
    for X_shuffled in Xs:
        res.append(-score_func(X_shuffled, y) + base_score)
    return res

def metric(y_true, y_pred):
    metrics = []
    for i in range(y_pred.shape[1]):
        if y_true[:, i].sum() > 1:
            metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float)))
    return np.mean(metrics)   

perm_imp = np.zeros(train.shape[1])
all_res = []
for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=0, shuffle=True).split(train_targets)):
    print(f'Fold {n}')

    model = create_model(len(train.columns))
    checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
    model.fit(train.values[tr],
                  train_targets.values[tr],
                  validation_data=(train.values[te], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
    model.load_weights(checkpoint_path)
        
    def _score(X, y):
        pred = model.predict(X)
        return metric(y, pred)

    base_score, local_imp = get_score_importances(_score, train.values[te], train_targets.values[te], n_iter=1, random_state=0)
    all_res.append(local_imp)
    perm_imp += np.mean(local_imp, axis=0)
    print('')
    
top_feats = np.argwhere(perm_imp < 0).flatten()
top_feats

Fold 0
Epoch 1/35
147/147 - 6s - loss: 0.6020 - val_loss: 0.2935
Epoch 2/35
147/147 - 1s - loss: 0.1565 - val_loss: 0.0618
Epoch 3/35
147/147 - 1s - loss: 0.0468 - val_loss: 0.0332
Epoch 4/35
147/147 - 1s - loss: 0.0303 - val_loss: 0.0262
Epoch 5/35
147/147 - 1s - loss: 0.0253 - val_loss: 0.0229
Epoch 6/35
147/147 - 1s - loss: 0.0227 - val_loss: 0.0219
Epoch 7/35
147/147 - 1s - loss: 0.0215 - val_loss: 0.0210
Epoch 8/35
147/147 - 1s - loss: 0.0207 - val_loss: 0.0210
Epoch 9/35
147/147 - 1s - loss: 0.0200 - val_loss: 0.0200
Epoch 10/35
147/147 - 1s - loss: 0.0194 - val_loss: 0.0195
Epoch 11/35
147/147 - 1s - loss: 0.0189 - val_loss: 0.0192
Epoch 12/35
147/147 - 1s - loss: 0.0186 - val_loss: 0.0189
Epoch 13/35
147/147 - 1s - loss: 0.0183 - val_loss: 0.0188
Epoch 14/35
147/147 - 1s - loss: 0.0181 - val_loss: 0.0185
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0184
Epoch 16/35
147/147 - 1s - loss: 0.0177 - val_loss: 0.0183
Epoch 17/35
147/147 - 1s - loss: 0.0174 - val_loss: 0.0181



Fold 1
Epoch 1/35
147/147 - 5s - loss: 0.6074 - val_loss: 0.3174
Epoch 2/35
147/147 - 1s - loss: 0.1600 - val_loss: 0.0624
Epoch 3/35
147/147 - 1s - loss: 0.0483 - val_loss: 0.0337
Epoch 4/35
147/147 - 1s - loss: 0.0308 - val_loss: 0.0251
Epoch 5/35
147/147 - 2s - loss: 0.0253 - val_loss: 0.0227
Epoch 6/35
147/147 - 1s - loss: 0.0230 - val_loss: 0.0213
Epoch 7/35
147/147 - 1s - loss: 0.0216 - val_loss: 0.0200
Epoch 8/35
147/147 - 1s - loss: 0.0208 - val_loss: 0.0194
Epoch 9/35
147/147 - 1s - loss: 0.0201 - val_loss: 0.0189
Epoch 10/35
147/147 - 1s - loss: 0.0195 - val_loss: 0.0185
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0182
Epoch 12/35
147/147 - 1s - loss: 0.0188 - val_loss: 0.0181
Epoch 13/35
147/147 - 1s - loss: 0.0185 - val_loss: 0.0178
Epoch 14/35
147/147 - 1s - loss: 0.0183 - val_loss: 0.0177
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0174
Epoch 16/35
147/147 - 1s - loss: 0.0181 - val_loss: 0.0175
Epoch 17/35
147/147 - 1s - loss: 0.0175 - val_loss: 0.01



Fold 2
Epoch 1/35
147/147 - 4s - loss: 0.6040 - val_loss: 0.3045
Epoch 2/35
147/147 - 1s - loss: 0.1587 - val_loss: 0.0631
Epoch 3/35
147/147 - 1s - loss: 0.0476 - val_loss: 0.0345
Epoch 4/35
147/147 - 1s - loss: 0.0307 - val_loss: 0.0256
Epoch 5/35
147/147 - 1s - loss: 0.0255 - val_loss: 0.0232
Epoch 6/35
147/147 - 1s - loss: 0.0231 - val_loss: 0.0214
Epoch 7/35
147/147 - 1s - loss: 0.0214 - val_loss: 0.0204
Epoch 8/35
147/147 - 1s - loss: 0.0206 - val_loss: 0.0196
Epoch 9/35
147/147 - 1s - loss: 0.0203 - val_loss: 0.0201
Epoch 10/35
147/147 - 1s - loss: 0.0196 - val_loss: 0.0189
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0187
Epoch 12/35
147/147 - 1s - loss: 0.0188 - val_loss: 0.0184
Epoch 13/35
147/147 - 1s - loss: 0.0185 - val_loss: 0.0180
Epoch 14/35
147/147 - 1s - loss: 0.0182 - val_loss: 0.0180
Epoch 15/35
147/147 - 1s - loss: 0.0180 - val_loss: 0.0178
Epoch 16/35
147/147 - 1s - loss: 0.0176 - val_loss: 0.0175
Epoch 17/35
147/147 - 2s - loss: 0.0175 - val_loss: 0.01



Fold 3
Epoch 1/35
147/147 - 5s - loss: 0.6051 - val_loss: 0.3053
Epoch 2/35
147/147 - 1s - loss: 0.1598 - val_loss: 0.0663
Epoch 3/35
147/147 - 1s - loss: 0.0473 - val_loss: 0.0318
Epoch 4/35
147/147 - 1s - loss: 0.0309 - val_loss: 0.0240
Epoch 5/35
147/147 - 1s - loss: 0.0252 - val_loss: 0.0218
Epoch 6/35
147/147 - 1s - loss: 0.0229 - val_loss: 0.0202
Epoch 7/35
147/147 - 1s - loss: 0.0216 - val_loss: 0.0195
Epoch 8/35
147/147 - 1s - loss: 0.0208 - val_loss: 0.0190
Epoch 9/35
147/147 - 1s - loss: 0.0206 - val_loss: 0.0186
Epoch 10/35
147/147 - 1s - loss: 0.0197 - val_loss: 0.0182
Epoch 11/35
147/147 - 1s - loss: 0.0195 - val_loss: 0.0180
Epoch 12/35
147/147 - 1s - loss: 0.0189 - val_loss: 0.0176
Epoch 13/35
147/147 - 1s - loss: 0.0186 - val_loss: 0.0174
Epoch 14/35
147/147 - 1s - loss: 0.0183 - val_loss: 0.0172
Epoch 15/35
147/147 - 1s - loss: 0.0180 - val_loss: 0.0171
Epoch 16/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0170
Epoch 17/35
147/147 - 1s - loss: 0.0176 - val_loss: 0.01



Fold 4
Epoch 1/35
147/147 - 4s - loss: 0.6044 - val_loss: 0.3087
Epoch 2/35
147/147 - 2s - loss: 0.1580 - val_loss: 0.0648
Epoch 3/35
147/147 - 1s - loss: 0.0475 - val_loss: 0.0332
Epoch 4/35
147/147 - 2s - loss: 0.0304 - val_loss: 0.0258
Epoch 5/35
147/147 - 1s - loss: 0.0255 - val_loss: 0.0229
Epoch 6/35
147/147 - 1s - loss: 0.0232 - val_loss: 0.0222
Epoch 7/35
147/147 - 2s - loss: 0.0218 - val_loss: 0.0202
Epoch 8/35
147/147 - 1s - loss: 0.0207 - val_loss: 0.0199
Epoch 9/35
147/147 - 1s - loss: 0.0200 - val_loss: 0.0192
Epoch 10/35
147/147 - 1s - loss: 0.0197 - val_loss: 0.0188
Epoch 11/35
147/147 - 1s - loss: 0.0192 - val_loss: 0.0186
Epoch 12/35
147/147 - 1s - loss: 0.0189 - val_loss: 0.0184
Epoch 13/35
147/147 - 1s - loss: 0.0186 - val_loss: 0.0181
Epoch 14/35
147/147 - 1s - loss: 0.0183 - val_loss: 0.0181
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0178
Epoch 16/35
147/147 - 1s - loss: 0.0177 - val_loss: 0.0177
Epoch 17/35
147/147 - 1s - loss: 0.0175 - val_loss: 0.01



Fold 5
Epoch 1/35
147/147 - 5s - loss: 0.6036 - val_loss: 0.3155
Epoch 2/35
147/147 - 1s - loss: 0.1582 - val_loss: 0.0632
Epoch 3/35
147/147 - 1s - loss: 0.0470 - val_loss: 0.0338
Epoch 4/35
147/147 - 1s - loss: 0.0308 - val_loss: 0.0283
Epoch 5/35
147/147 - 1s - loss: 0.0252 - val_loss: 0.0227
Epoch 6/35
147/147 - 1s - loss: 0.0231 - val_loss: 0.0218
Epoch 7/35
147/147 - 1s - loss: 0.0215 - val_loss: 0.0202
Epoch 8/35
147/147 - 1s - loss: 0.0206 - val_loss: 0.0199
Epoch 9/35
147/147 - 1s - loss: 0.0200 - val_loss: 0.0193
Epoch 10/35
147/147 - 1s - loss: 0.0196 - val_loss: 0.0191
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0186
Epoch 12/35
147/147 - 1s - loss: 0.0188 - val_loss: 0.0184
Epoch 13/35
147/147 - 1s - loss: 0.0184 - val_loss: 0.0181
Epoch 14/35
147/147 - 1s - loss: 0.0182 - val_loss: 0.0180
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0177
Epoch 16/35
147/147 - 1s - loss: 0.0176 - val_loss: 0.0176
Epoch 17/35
147/147 - 1s - loss: 0.0174 - val_loss: 0.01



Fold 6
Epoch 1/35
147/147 - 5s - loss: 0.6015 - val_loss: 0.3066
Epoch 2/35
147/147 - 1s - loss: 0.1579 - val_loss: 0.0593
Epoch 3/35
147/147 - 1s - loss: 0.0475 - val_loss: 0.0337
Epoch 4/35
147/147 - 1s - loss: 0.0306 - val_loss: 0.0252
Epoch 5/35
147/147 - 1s - loss: 0.0252 - val_loss: 0.0227
Epoch 6/35
147/147 - 1s - loss: 0.0229 - val_loss: 0.0212
Epoch 7/35
147/147 - 1s - loss: 0.0215 - val_loss: 0.0204
Epoch 8/35
147/147 - 1s - loss: 0.0207 - val_loss: 0.0196
Epoch 9/35
147/147 - 1s - loss: 0.0199 - val_loss: 0.0190
Epoch 10/35
147/147 - 1s - loss: 0.0195 - val_loss: 0.0188
Epoch 11/35
147/147 - 1s - loss: 0.0191 - val_loss: 0.0183
Epoch 12/35
147/147 - 1s - loss: 0.0189 - val_loss: 0.0182
Epoch 13/35
147/147 - 1s - loss: 0.0185 - val_loss: 0.0180
Epoch 14/35
147/147 - 1s - loss: 0.0182 - val_loss: 0.0180
Epoch 15/35
147/147 - 1s - loss: 0.0179 - val_loss: 0.0177
Epoch 16/35
147/147 - 1s - loss: 0.0177 - val_loss: 0.0175
Epoch 17/35
147/147 - 1s - loss: 0.0175 - val_loss: 0.01

array([  1,   2,   3,   4,   5,   6,   7,   9,  10,  11,  14,  15,  16,
        17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,
        31,  32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,
        60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,
        73,  74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127,
       128, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 19

In [24]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [19]:
model = create_model(785)

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 785)               3140      
_________________________________________________________________
dropout (Dropout)            (None, 785)               0         
_________________________________________________________________
weight_normalization (Weight (None, 2048)              3221505   
_________________________________________________________________
batch_normalization_1 (Batch (None, 2048)              8192      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
weight_normalization_1 (Weig (None, 1048)              4295753   
_________________________________________________________________
batch_normalization_2 (Batch (None, 1048)              4

In [25]:
N_STARTS = 7
tf.random.set_seed(42)

res = train_targets.copy()
submission.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0

for seed in range(N_STARTS): #MultilabelstratifiedKFold 분포 맞춰서 KFold
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train_targets, train_targets)):
        print(f'Fold {n}')
    
        model = create_model(len(top_feats))
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[tr][:, top_feats],
                  train_targets.values[tr],
                  validation_data=(train.values[te][:, top_feats], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        submission.loc[:, train_targets.columns] += test_predict
        res.loc[te, train_targets.columns] += val_predict
        print('')
    
submission.loc[:, train_targets.columns] /= ((n+1) * N_STARTS)
res.loc[:, train_targets.columns] /= N_STARTS

NameError: ignored